# Data Mining – Assignment 1
---
> Chalkiopoulos Georgios, Electrical and Computer Engineer NTUA <br />
> Data Science postgraduate Student <br />
> gchalkiopoulos@aueb.gr

# Imports

In [5]:
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import csv

# 1) Import and pre-process the dataset with users

There are 3 files for the dataset,
* the users.txt file contains id, age, gender, occupation and postcode separated by |,
* the movies.txt file contains id, title (with release year) and some other information not related with the assignment separated by |,
* the ratings.txt file (tab separated) which contains userid, movieid, rating (1-5) and timestamp.

For this assignment only the set of movies that a user has rated, and not the ratings, will be used. In your report you should describe in detail any processing and conversion you made to the original data and the reasons it was necessary.

In [20]:
# Define phat
path = Path.cwd() / "MovieLensDataset"

In [29]:
# Users
users_col: list = ["id", "age", "gender", "occupation", "postcode"]
users_df: DataFrame = pd.read_csv(path / "users.txt", delimiter="|",names=users_col, header=None)
users_df.head()

,id,age,gender,occupation,postcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [24]:
# Movies
movies_col: list = ["id", "title_year"]
movies_df: DataFrame = pd.read_csv(path / "movies.txt", delimiter="|", encoding='latin-1', header=None, names=movies_col, usecols=movies_col)
movies_df.head()

,id,title_year
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [31]:
# Ratings
ratings_col: list = ["userid", "movieid", "rating", "timestamp"]
ratings_df: DataFrame = pd.read_csv(path / "ratings.txt", delimiter="\t", header=None, names=ratings_col)
ratings_df.head()

,userid,movieid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
